<a href="https://colab.research.google.com/github/Yogananth-r/situation-awareness/blob/main/Tank_Detection_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking
%cd Military_Vehicles_Tracking/

# download the trained model's weights
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/one_class_YOLOV5s.pt -P weights/
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/one_class_YOLOv5x.pt -P weights/
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/person_military_seperate_YOLOv5l.pt -P weights/

# download test videos
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/test1.mp4 -P test_videos/
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/test3.mp4 -P test_videos/
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/test4.mp4 -P test_videos/
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/test5.mp4 -P test_videos/
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/test10.mp4 -P test_videos/
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/test13.mp4 -P test_videos/
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/test16.mp4 -P test_videos/
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/test17.mp4 -P test_videos/

Cloning into 'Military_Vehicles_Tracking'...
remote: Enumerating objects: 241, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 241 (delta 5), reused 3 (delta 3), pack-reused 233
Receiving objects: 100% (241/241), 98.49 MiB | 33.02 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/Military_Vehicles_Tracking


In [ ]:
import io
import os
import torch
from pprint import pprint
from base64 import b64encode
import matplotlib.pyplot as plt
from IPython.display import HTML, clear_output

from Military_Vehicles_Tracking.src.video import Video
from Military_Vehicles_Tracking.src.tracker import MultiObjectTracker
from Military_Vehicles_Tracking.src.plot_utils import plot_bounding_boxes

!pip install -qq --ignore-installed PyYAML

# setup GPU device if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Runtime details:')
print(f'torch {torch.__version__}')
print(torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU')

# get available weights
weights_path = 'weights'
weights_names = os.listdir(weights_path)
weights = {name: f'{weights_path}/{name}' for name in weights_names}

# get available test videos
test_videos_path = 'test_videos'
for i, video_name in enumerate(sorted(os.listdir(test_videos_path))):
    os.rename(f'{test_videos_path}/{video_name}', f'{test_videos_path}/test_{i+1}.mp4')
for i, video_name in enumerate(sorted(os.listdir(test_videos_path))):
    os.rename(f'{test_videos_path}/{video_name}', f'{test_videos_path}/test{i+1}.mp4')
videos_names = sorted(os.listdir(test_videos_path))
videos_paths = [f'{test_videos_path}/{name}' for name in videos_names]

# create folders for the results
for weights_name, weights_path in weights.items():
    os.makedirs(f'results/{weights_name}', exist_ok=True)

print('\nAvailable trained weights:')
pprint(list(weights.values()))

print('\nAvailable test videos:')
pprint(videos_paths)

/content/Military_Vehicles_Tracking/src/video.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.8/661.8 KB 12.7 MB/s eta 0:00:00
Runtime details:
torch 2.0.0+cu118
Tesla T4

Available trained weights:
['weights/person_military_seperate_YOLOv5l.pt',
 'weights/one_class_YOLOV5s.pt',
 'weights/one_class_YOLOv5x.pt']

Available test videos:
['test_videos/test1.mp4',
 'test_videos/test2.mp4',
 'test_videos/test3.mp4',
 'test_videos/test4.mp4',
 'test_videos/test5.mp4',
 'test_videos/test6.mp4',
 'test_videos/test7.mp4',
 'test_videos/test8.mp4']


In [ ]:
detectors = [
    torch.hub.load('ultralytics/yolov5', 'custom', weights_path).to(device) 
    for weights_path in weights.values()
]

/usr/local/lib/python3.9/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: YOLOv5 requirement "gitpython>=3.1.30" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 5.

In [ ]:
video = "test7.mp4" #@param ["test1.mp4", "test2.mp4", "test3.mp4", "test4.mp4", "test5.mp4", "test6.mp4", "test7.mp4", "test8.mp4"]
detector = 'One class YOLOv5s' #@param ["One class YOLOv5s", "One class YOLOv5x", "Multiple class YOLOv5l"]

video_name = video
detector_map = {"One class YOLOv5s": 0,
                "One class YOLOv5x": 1,
                "Multiple class YOLOv5l": 2}
detector_idx = detector_map[detector]

video = Video(f'{test_videos_path}/{video_name}')

weights_name = list(weights.keys())[detector_idx]
detector = detectors[detector_idx]

results_path = f'results/{weights_name}/{video_name}'
tracker = MultiObjectTracker(video, results_path, detector)


for frame, bounding_boxes in tracker:
    plot_bounding_boxes(frame, bounding_boxes)
    tracker.video_writer.write(frame)


tracker.video_writer.release()  # saving the results
print(f'\n\n=> ({weights_name}) - Saved results for: {video_name}')
print(f'   result saved in {results_path}')


print('\nConverting video to displayable format...')
path_webm = results_path.replace('mp4', 'webm')
!ffmpeg -loglevel warning -y -i "$results_path" -vcodec vp9 "$path_webm"


with  io.open(path_webm,'r+b') as f:
    vid = f.read()


data_url = "data:video/webm;base64," + b64encode(vid).decode()  # displaying video
HTML("""
<video width=800 controls>
    <source src="%s" type="video/webm">
</video>
""" % data_url)

Tracking: 100%|██████████| 357/357 [00:36<00:00,  9.69frame/s]




=> (person_military_seperate_YOLOv5l.pt) - Saved results for: test7.mp4
   result saved in results/person_military_seperate_YOLOv5l.pt/test7.mp4

Converting video to displayable format...
